In [ ]:
*Note about gcloud config:*
- there are four type of operations in gcloud config: get, list, set, unset.
- before running any config command, we have to create project so that we have project_id to work with.
- to set project in core section use command: gcloud config set project project_id.

*Note about gcloud container:*
- gclouc container have multiples resource points: aws, azure, cluters. in google cloud we use clusters resource end point.
- gcloud container clusters have operations like: create, create-auto, delete, list

In [ ]:
#example: create cluster with name airflow-cluster and some other config
#before start clustter enable kubernetes api by go to api & service page and click enable api and service, search for kubernetes engine api
gcloud container clusters create airflow-cluster \
--machine-type n1-standard-4 \
--num-nodes 1 \
--region "asia-southeast1-c"

In [ ]:
#authenticate kubectl to access and control this cluster
gcloud container clusters get-credentials airflow-cluster --region "asia-southeast1-c"

In [ ]:
*Note about kubectl:*
- syntax: kubectl [command] [TYPE] [NAME] [flags]
- example: kubectl get pod pod1kubectl create namespace airflow
- to create name space: kubectl create namespace airflow

In [ ]:
#Note about heml:


#add airflow helm chart to your local computer with command:
helm repo add apache-airflow https://airflow.apache.org
#check local hem chart with command:
helm repo list
#deploy kubernetes with helm install command, and debug flag
helm install airflow apache-airflow/airflow --namespace airflow --debug
# deploy airflow with helm with command:
helm upgrade --install airflow apache-airflow/airflow -n airflow --debug
# syntax of helm upgrade: 
helm upgrade [release] [chart] [flag]. with flag --install for the first name, -n for name space and --debug to view install process.
# check the installed pods with kubectl get pod command:
# this command will show which pod is running and which pod is eror, useful for debug
kubectl get pod -n name_space
#check for the service with command
#check the install compoents by go to: service and ingress section.
kubectl get service -n name_space
#by default service is access inside cluster only. to access service from outside of cluster we use command port-forward
#forwad port from kebernetes port to localhost port
kubectl port-forward svc/airflow-webserver 8080:8080 -n airflow
- save the content of the current helm chart with command: helm show values apache-airflow/airflow > values.yaml.
- custom your chart file with change in executor and service type from ClusterIp to LoadBalancer.
- reinstall the custom chart with command: helm upgrade --install airflow apache-airflow/airflow -n airflow  \
  -f values.yaml \
  --debug

In [ ]:
#note about sync between github or gitlab repo with airflow webserver dag,
#have to use email with flag C
- generate ssh-key with command ssh-keygen: ssh-keygen -t rsa -b 4096 -C "toanbui1991@gmail.com"

In [ ]:
kubectl create secret generic airflow-git-ssh-secret \
  --from-file=gitSshKey=/root/.ssh/id_rsa \
  -n airflow

In [ ]:
#create webserver secrete key:
kubectl create secret generic airflow-webserver-secret --from-literal="webserver-secret-key=$(python3 -c 'import secrets; print(secrets.token_hex(16))')"
#add config at: webserverSecretKeySecretName: my-webserver-secret

In [ ]:
#update deployment in kubernetes with command:
helm upgrade --install airflow apache-airflow/airflow -n airflow -f values.yaml --debug

In [ ]:
#add private key to kubernetes cluster so that kubernete clustter can async with remote repo
kubectl create secret generic airflow-gke-git-secret --from-file=gitSshKey=/root/.ssh/id_rsa -n airflow


In [ ]:
#delete cluster with command:
gcloud container clusters delete cluster_name
#example: 
gcloud container clusters delete airflow-cluster --zone "asia-southeast1-c"


In [ ]:
#update kubernete deployement:
helm upgrade --install airflow apache-airflow/airflow -n airflow -f values.yaml --debug
helm upgrade --install airflow apache-airflow/airflow -n airflow -f values.yaml --debug

In [ ]:
#create kubectl secrete so that cluster can access git repo
kubectl create secret generic airflow-ssh-git-secret \
  --from-file=gitSshKey=/home/toanbui1991/.ssh/id_rsa \
  --from-file=known_hosts=/home/toanbui1991/.ssh/known_hosts \
  --from-file=id_rsa.pub=/home/toanbui1991/.ssh/id_rsa.pub \
  -n airflow

kubectl create secret generic airflow-git-ssh-secret \
    --from-file=ssh-privatekey=/home/toanbui1991/.ssh/id_rsa \
    --from-file=ssh-publickey=/home/toanbui1991/.ssh/id_rsa.pub

In [ ]:
#Note about why we can not sync dags with kubernets cluster
#one wrong wrong ssh address because do not change ssh address to required form
#have to change from : to / after git@github.com
#git@github.com:toanbui1991/airflow_kubernetes_v2.git, change to, ssh://git@github.com/toanbui1991/airflow_kubernetes_v2.git
#second, kubernet do not see secrete which is use to authenticate git repo
#three, we have to check and make sure secret can be seen by kubernete cluster with the same namespace

#link for reference one: https://blog.devgenius.io/setting-up-apache-airflow-on-kubernetes-with-gitsync-beaac2e397f3
#link for reference two: https://stackoverflow.com/questions/62167682/issue-setting-up-git-sync-with-airflow-in-a-pod

#we also have to know how to debug pods
    # kubectl get pod -n namespace
    # kubectl get service -n namespace
    # kubectl logs pod_id -n namespace
    # kubectl logs pod_id log_name -n namespace
    # kubectl describe pod_id -n namespace

In [ ]:
#Note about how to solve other warning message
#one set up static webserver secret key to help airflow component only restart when necessary
    #with this link: https://airflow.apache.org/docs/helm-chart/stable/production-guide.html#webserver-secret-key
#two if you use dags.gitSync.sshKeySecret, you should also use dags.gitSync.knownHosts
    #step one, get github public key with command: ssh-keyscan -t rsa github.com > github_public_key
    #step two, get public key fingerprint with command: ssh-keygen -lf github_public_key
    #and comapre the result from step two, with github public key
    #step three: add github public key to values.ymal file.